## Формат DateTime
В Pandas есть формат DateTime, который записывается как YYYY-MM-DD HH: MM: SS
В наших данных формат даты указывается как DD-MM-YYYY

In [1]:
import pandas as pd

In [8]:
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

In [9]:
# Для того, чтобы преобразовывать столбцы с датами, записанными в распространенных форматах, в формат datetime, можно воспользоваться функцией
# pandas.to_datetime(). В данном случае нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке
# первым идет день.
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

# После преобразования тип данных столбца Date изменился на datetime64 

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

## Выделение атрибутов Datetime
Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:
- date - дата
- year, month, day - год, месяц, день
- time - время
- hour, minute, seconds
- dayofweek
- day_name
- dayofyear - порядковый день года
- quarter - квартал
Например, обратившись по атрибуту dt.year в столбце Date, можно достать год продажи для каждого элемента и понять, за какой интервал времени представлены наши данные, а также на какой год приходится наибольшее число продаж 

In [12]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold: ', years_sold.min())
print('Max year sold: ', years_sold.max())
print('Mode year sold: ', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int32
Min year sold:  2016
Max year sold:  2017
Mode year sold:  2017


## Теперь попробуем понять на какой месяц приходится пик продаж недвижимости.
Для этого выделим атрибут dt.month и на этот раз занесем данные в столбец MonthSale, а затем найдем относительную частоту продаж для каждого месяца от общего количества продаж - для этого используется метод value_counts() с параметром normalize (вывод в долях)

In [13]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

MonthSale
5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: proportion, dtype: float64

## Работа с интервалами
Часто бывает ситуация, когда необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime

In [14]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
display(delta_days)

## Даты такого формата относятся к типа timedelta

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

In [15]:
## Чтобы превратить количество дней из формата интервала в формат целого числа, можно воспользоваться
# аксессором dt для формата timedelta и извлечь из него атрибут days

display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

In [16]:
# Создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой год и вычтем из него
# год постройки. Результат занесем в столбец AgeBuilding
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

По сути столбец AgeBuilding дублирует данные из столбца YearBuilt, только в более удобном формате, поэтому можно смело убрать YearBuilt 

In [17]:
melb_df = melb_df.drop('YearBuilt', axis=1)

In [ ]:
#Создайте в таблице melb_df признак WeekdaySale (день недели). Найдите, сколько объектов недвижимости было продано в выходные 
#(суббота и воскресенье), результат занесите в переменную weekend_count. В качестве ответа введите результат вывода переменной weekend_count.

melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

12822


In [30]:
nlo = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv')
nlo['Time'] = pd.to_datetime(nlo['Time'])
print(nlo['Time'].dt.year.mode())

0    1999
Name: Time, dtype: int32


In [31]:
nlo['Time'].dt.date

0        1930-06-01
1        1930-06-30
2        1931-02-15
3        1931-06-01
4        1933-04-18
            ...    
18236    2000-12-31
18237    2000-12-31
18238    2000-12-31
18239    2000-12-31
18240    2000-12-31
Name: Time, Length: 18241, dtype: object